In [ ]:
!python -m pip install \
umap \
matplotlib \
scikit-learn \
seaborn \
umap-learn \
sentence-transformers

In [ ]:
data_path = "data"

In [ ]:
!wget -P {data_path} -nc "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

In [ ]:
!tar -zxf {data_path}/ldcc-20140209.tar.gz -C {data_path}

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sentence_transformers import SentenceTransformer
import umap.umap_ as umap
import time
%matplotlib inline

In [ ]:
def extract_contents(file_name):
    with open(file_name) as file:
        lines = file.readlines()
    title = lines[2].strip()
    contents = lines[3:]
    contents = [sentence.strip() for sentence in contents]
    contents = list(filter(lambda line: line != '', contents))
    contents = [l.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': ''}))
        for l in contents]
    contents = ["".join(contents)]
    return title, contents

In [ ]:
contents_path = os.path.join(data_path, "text")
file_name = os.path.join(contents_path, "dokujo-tsushin", "dokujo-tsushin-4778030.txt")
extract_contents(file_name)

In [ ]:
data_list = []
categories = []

for cat in os.listdir(contents_path):
    if not os.path.isdir(os.path.join(contents_path, cat)):
        continue
    print(cat)
    categories.append(cat)
    files = glob.glob(os.path.join(contents_path, cat, "*.txt"))
    for f in files:
        title, contents = extract_contents(f)
        if title.startswith("原著作者のクレジットを表示し、"):
            continue
        data_list += [[cat, title, cnt] for cnt in contents]

In [ ]:
len(data_list)

In [ ]:
data_list[2]

In [ ]:
data_frame = pd.DataFrame(data_list, columns=["category", "title", "sentence"])

In [ ]:
data_frame.head()

In [ ]:
data = data_frame["sentence"]

In [ ]:
model = SentenceTransformer(
    'sentence-transformers/distiluse-base-multilingual-cased-v2')

In [ ]:
embeddings = model.encode(data, show_progress_bar=True)
embeddings.shape

In [ ]:
reducer = umap.UMAP()

In [ ]:
stime = time.time()
mapped_data = reducer.fit_transform(embeddings)
print(time.time() - stime)
mapped_data.shape

In [ ]:
data_frame["umap_x"] = mapped_data[:, 0]
data_frame["umap_y"] = mapped_data[:, 1]

In [ ]:
data_frame.head()

In [ ]:
cat_data = data_frame["category"]
numbers = range(len(categories))
color_dict = {cat: num for cat, num in zip(categories, numbers)}

In [ ]:
fig, ax = plt.subplots()

for cat in categories:
    color = sns.color_palette()[color_dict[cat]]
    df = data_frame[data_frame["category"] == cat]
    x = df["umap_x"].values
    y = df["umap_y"].values
    n = len(x)
    scatter = ax.scatter(
        x, y,
        color=color,
        label=cat,
        marker="."
    )
    print(f"{cat} {n}")

ax.legend()
ax.grid(True)
plt.gca().set_aspect("equal", "datalim")
plt.title("UMAP projection of the livedoor news corpus", fontsize=20)
plt.show()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

In [ ]:
num_figs = len(categories)
num_cols = 3
num_rows = (num_figs + num_cols - 1) // num_cols

fig, axs = plt.subplots(ncols=num_cols, nrows=num_rows, figsize=(9, 6),
                        layout="constrained")

i = 0
for row in range(num_rows):
    for col in range(num_cols):
        cat = categories[i]
        color = sns.color_palette()[color_dict[cat]]
        df = data_frame[data_frame["category"] == cat]
        x = df["umap_x"].values
        y = df["umap_y"].values
        ax = axs[row, col]
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.grid(True)
        ax.scatter(
            x, y,
            color=color,
            label=cat,
            marker="."
        )
        ax.annotate(
            f"{cat}", (xlim[0], ylim[1]),
            transform=axs[row, col].transAxes,
            ha='left', va='top', fontsize=18,
            color='darkgrey'
        )
        i += 1
fig.suptitle("UMAP projection of the livedoor news corpus")

In [ ]:
data_frame[(data_frame["category"] == "smax") & (data_frame["umap_y"] > 8.0)]

In [ ]:
data_frame[(data_frame["category"] == "sports-watch") & (data_frame["umap_y"] < 5.0)]